In [6]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit_algorithms.optimizers import COBYLA
import itertools
from collections import Counter

# 参数定义
n = 2  # 作业数量
m = 3  # 机器数量
N = int(np.ceil(np.log2(m)))  # 二进制位数表示机器号
l = 3  # C_max 的二进制位数
p = np.array([[1, 2, 3], [2, 3, 1]])  # 处理时间矩阵
b = np.array([[0, 0], [1, 0], [0, 1]])  # 机器的二进制表示 (0, 1, 2)
W = [2]  # 不可用机器集合 (机器 2，对应 [0, 1])
K1 = 100  # 惩罚系数 1
P = 200  # 惩罚系数 2
lambda_penalty = 1000  # 降阶惩罚系数
num_qubits_base = n * N + l  # 原始量子比特数 (7)


# 计算辅助比特需求
def count_aux_needed(n, m, N, l, W):
    aux_per_term = max(0, N - 2)  # 每个多体项 (k > 2) 需要的辅助比特
    h2_terms = len(W) * n * (2 ** N)  # H_2 多体项数
    h3_cj2_terms = m * n * n * (2 ** N) * (2 ** N)  # H_3 C_j^2 项
    h3_cj_cmax_terms = m * n * (2 ** N) * l  # H_3 -2 C_j C_max 项
    total_terms = h2_terms + h3_cj2_terms + h3_cj_cmax_terms
    return total_terms * aux_per_term


num_aux_qubits = count_aux_needed(n, m, N, l, W)
num_qubits = num_qubits_base + num_aux_qubits
print(f"Total qubits: {num_qubits}, Base: {num_qubits_base}, Aux: {num_aux_qubits}")


# 定义量子门函数
def append__z_term(qc, qubit, gamma):
    qc.rz(2 * gamma, qubit)


def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__mixer_term(qc, qubit, beta):
    qc.rx(2 * beta, qubit)


# 降阶多体项函数
def append_reduced_multi_rzz_term(qc, qubits, gamma, aux_start_idx, max_aux_idx):
    """将多体 Z 项降阶为双体项，返回使用的辅助比特数"""
    if len(qubits) <= 2:
        if len(qubits) == 1:
            append__z_term(qc, qubits[0], gamma)
        elif len(qubits) == 2:
            append__zz_term(qc, qubits[0], qubits[1], gamma)
        return 0
    else:
        current_qubits = list(qubits)
        aux_used = 0
        while len(current_qubits) > 2:
            if aux_start_idx + aux_used >= max_aux_idx:
                raise ValueError(f"Auxiliary qubit index {aux_start_idx + aux_used} exceeds max {max_aux_idx}")
            q1, q2 = current_qubits[:2]
            aux_idx = aux_start_idx + aux_used
            coef = gamma if aux_used == 0 else lambda_penalty
            append__zz_term(qc, q1, q2, coef / 2)
            append__z_term(qc, aux_idx, -coef / 2)
            append__zz_term(qc, q1, aux_idx, -lambda_penalty / 2)
            append__zz_term(qc, q2, aux_idx, -lambda_penalty / 2)
            append__z_term(qc, q1, lambda_penalty / 4)
            append__z_term(qc, q2, lambda_penalty / 4)
            append__z_term(qc, aux_idx, lambda_penalty / 4)
            current_qubits = [aux_idx] + current_qubits[2:]
            aux_used += 1
        if len(current_qubits) == 2:
            append__zz_term(qc, current_qubits[0], current_qubits[1], gamma)
        elif len(current_qubits) == 1:
            append__z_term(qc, current_qubits[0], gamma)
        return aux_used


# H_1: -1/2 ∑_{h=0}^{l-1} 2^h Z_h
def get_cost_circuit_h1(gamma, qc):
    for h in range(l):
        coef = -0.5 * 2 ** h
        append__z_term(qc, n * N + h, gamma * coef)
    return qc


# H_2: K_1 ∑_{j∈W} ∑_{i=0}^{n-1} 1/2^N ∑_{S} (∏_{k∈S} (1 - 2 b_{jk})) (∏_{k∈S} Z_{ik})
def get_cost_circuit_h2(gamma, qc, aux_start_idx, max_aux_idx):
    all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(N + 1))))
    aux_idx = aux_start_idx
    for j in W:
        for i in range(n):
            for S in all_combinations:
                coef = K1 / (2 ** N)
                qubits = []
                for k in S:
                    coef *= (1 - 2 * b[j, k])
                    qubits.append(i * N + k)
                aux_used = append_reduced_multi_rzz_term(qc, qubits, gamma * coef, aux_idx, max_aux_idx)
                aux_idx += aux_used
    return qc, aux_idx


# H_3: P ∑_{j=0}^{m-1} (C_j - C_max)^2
def get_cost_circuit_h3(gamma, qc, aux_start_idx, max_aux_idx):
    all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(N + 1))))
    aux_idx = aux_start_idx

    for j in range(m):
        # C_j^2
        for i in range(n):
            for i_prime in range(n):
                for S_i in all_combinations:
                    for S_i_prime in all_combinations:
                        coef = P * p[i, j] * p[i_prime, j] / (2 ** (2 * N))
                        qubits = []
                        for k in S_i:
                            coef *= (1 - 2 * b[j, k])
                            qubits.append(i * N + k)
                        for k in S_i_prime:
                            coef *= (1 - 2 * b[j, k])
                            qubits.append(i_prime * N + k)
                        count = Counter(qubits)
                        qubits_final = [q for q, cnt in count.items() if cnt % 2 == 1]
                        if len(qubits_final) > 0:
                            aux_used = append_reduced_multi_rzz_term(qc, qubits_final, gamma * coef, aux_idx,
                                                                     max_aux_idx)
                            aux_idx += aux_used

        # -2 C_j C_max
        for i in range(n):
            for S_i in all_combinations:
                coef_base = P * p[i, j] / (2 ** (N - 1))
                qubits_S = []
                for k in S_i:
                    coef_base *= (1 - 2 * b[j, k])
                    qubits_S.append(i * N + k)
                for h in range(l):
                    coef = coef_base * 2 ** h
                    aux_used = append_reduced_multi_rzz_term(qc, qubits_S, -2 * gamma * coef, aux_idx, max_aux_idx)
                    aux_idx += aux_used
                    qubits_combined = qubits_S + [n * N + h]
                    aux_used = append_reduced_multi_rzz_term(qc, qubits_combined, 2 * gamma * coef, aux_idx,
                                                             max_aux_idx)
                    aux_idx += aux_used

        # C_max^2
        for h in range(l):
            coef = -0.5 * P * (2 ** l - 1) * 2 ** h
            append__z_term(qc, n * N + h, gamma * coef)
            for h_prime in range(l):
                if h != h_prime:
                    coef = 0.25 * P * 2 ** h * 2 ** h_prime
                    append__zz_term(qc, n * N + h, n * N + h_prime, gamma * coef)

    return qc, aux_idx


# 混合器电路
def get_mixer_circuit(beta, qc):
    for i in range(num_qubits):
        append__mixer_term(qc, i, beta)
    return qc


# 反转比特序
def invert_counts(s):
    return s[::-1]


# 成本函数
def cost_function(params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    aux_idx = num_qubits_base
    for i in range(len(beta)):
        get_cost_circuit_h1(gamma[i], qc)
        qc, aux_idx_h2 = get_cost_circuit_h2(gamma[i], qc, aux_idx, num_qubits)
        qc, aux_idx = get_cost_circuit_h3(gamma[i], qc, aux_idx_h2, num_qubits)
    qc.measure(range(num_qubits), range(num_qubits))

    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)

    expectation = 0
    offset = P * m * (2 ** l - 1) ** 2 / 4  # H_3 的常数项
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[n * N:n * N + l]])
        s_ik = np.array([int(bit) for bit in bitstring[:n * N]]).reshape(n, N)

        E1 = np.sum([2 ** h * z_h[h] for h in range(l)])
        E2 = 0
        for j in W:
            for i in range(n):
                constant = 1
                for k in range(N):
                    constant *= (1 - (s_ik[i, k] - b[j, k]) ** 2)
                E2 += constant
        E2 = K1 * E2
        E3 = 0
        for j in range(m):
            load = 0
            for i in range(n):
                constant = 1
                for k in range(N):
                    constant *= (1 - (s_ik[i, k] - b[j, k]) ** 2)
                load += p[i, j] * constant
            c_max = np.sum([2 ** h * z_h[h] for h in range(l)])
            E3 += P * (load - c_max) ** 2

        E = E1 + E2 + E3
        expectation += E * prob

    return expectation + offset, counts


# 参数优化
def optimize_qaoa(params):
    def objective(params):
        expectation, _ = cost_function(params)
        return expectation

    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    result = optimizer.minimize(fun=objective, x0=params)
    return result


# 傅里叶参数化初始化
def fourier_initial_params(p, q=1, amplitude=0.5):
    t = np.linspace(0, 1, p)
    beta = np.zeros(p)
    gamma = np.zeros(p)
    for k in range(1, q + 1):
        beta += (np.random.uniform(0, amplitude) * np.cos(2 * np.pi * k * t) +
                 np.random.uniform(0, amplitude) * np.sin(2 * np.pi * k * t))
        gamma += (np.random.uniform(0, amplitude) * np.cos(2 * np.pi * k * t) +
                  np.random.uniform(0, amplitude) * np.sin(2 * np.pi * k * t))
    beta = np.clip(beta, 0, np.pi)
    gamma = np.clip(gamma, 0, 2 * np.pi)
    return np.concatenate([beta, gamma])


# 主程序
import time

startTime = time.time()
min_energy = float('inf')
init_point = np.array([])
counts = {}
final_result = None

for p1 in range(1, 4):
    min_energy = 100000
    for k in range(10):
        print(f"第 {p1} 层，第 {k} 个参数")
        init_point_temp = fourier_initial_params(p1, q=1, amplitude=0.5)

        result = optimize_qaoa(init_point_temp)
        optimal_params = result.x
        energy, counts_temp = cost_function(result.x)

        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
            print(f"New best energy: {min_energy}")
            print(f"Optimal parameters: {init_point}")

endTime = time.time()
print(f"Total time: {endTime - startTime} seconds")
print(f"Final minimum energy: {min_energy}")
print(f"Final optimal parameters: {final_result.x}")
print(f"Final counts: {counts}")

Total qubits: 7, Base: 7, Aux: 0
第 1 层，第 0 个参数


ValueError: Auxiliary qubit index 7 exceeds max 7

In [3]:
# 主程序
import time

startTime = time.time()
min_energy = float('inf')
init_point = np.array([])
counts = {}
final_result = None

for p1 in range(1, 4):
    min_energy = 100000
    for k in range(10):
        print(f"第 {p1} 层，第 {k} 个参数")
        if k == 0 and p1 == 1:
            init_point_temp = np.concatenate([np.random.uniform(0, np.pi, p1),
                                              np.random.uniform(0, 2 * np.pi, p1)])
        elif k == 0:
            # 使用插值法从前一层生成新参数
            init_point_temp = interpolate_parameters(init_point, p1 - 1, p1)
        else:
            # 随机调整当前层的参数
            init_point_temp = init_point.copy()
            beta_p = np.random.uniform(0, np.pi)
            gamma_p = np.random.uniform(0, 2 * np.pi)
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
            init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
            init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)

        result = optimize_qaoa(init_point_temp)
        optimal_params = result.x
        energy, counts_temp = cost_function(result.x)

        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
            print(f"New best energy: {min_energy}")
            print(f"Optimal parameters: {init_point}")

endTime = time.time()

第 1 层，第 0 个参数


IndexError: index 3 is out of bounds for axis 0 with size 3